<a href="https://colab.research.google.com/github/adamrusling/6G7V0033_2425_9F/blob/main/runtime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install libraries, required each time
!pip install eciespy
!pip install pycryptodomex

In [ ]:
import os
import json
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes

# Directory to store encrypted files when code is executed
ENC_DIR = "encrypted_files"
DEC_DIR = "decrypted_files"

# Generate RSA key pair
def generate_rsa_keys():
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    return private_key, public_key

# Encrypt file using AES-GCM and encrypt AES key using RSA
def encrypt_file(file_path, public_key):
    # 128-bit AES key
    aes_key = get_random_bytes(16)
    # 96-bit nonce for AES-GCM
    nonce = get_random_bytes(12)

    with open(file_path, "rb") as f:
        plaintext = f.read()

    cipher_aes = AES.new(aes_key, AES.MODE_GCM, nonce=nonce)
    ciphertext, tag = cipher_aes.encrypt_and_digest(plaintext)

    # Encrypt AES key using RSA
    rsa_key = RSA.import_key(public_key)
    cipher_rsa = PKCS1_OAEP.new(rsa_key)
    encrypted_aes_key = cipher_rsa.encrypt(aes_key)

    # Save encrypted data into a separate file
    enc_file_path = os.path.join(ENC_DIR, os.path.basename(file_path) + ".enc")
    metadata = {
        "nonce": nonce.hex(),
        "tag": tag.hex(),
        "enc_aes_key": encrypted_aes_key.hex()
    }
    with open(enc_file_path, "wb") as f:
        f.write(ciphertext)

    with open(enc_file_path + ".meta", "w") as f:
        # Convert data into a string to write output to a readable file
        json.dump(metadata, f)

    return enc_file_path

# Digital signature to verify file integrity
def sign_file(private_key, file_path):
    rsa_key = RSA.import_key(private_key)

    # Open the file path
    with open(file_path, "rb") as f:
        data = f.read()

    # Hash the file before generating the digital signature
    hash = SHA256.new(data)
    signature = pkcs1_15.new(rsa_key).sign(hash)

    # Write the signature to file
    with open(file_path + ".sig", "wb") as f:
        f.write(signature)

# Decrypt file
def decrypt_file(enc_file_path, private_key):
    # Read metadata to gather encryption components
    with open(enc_file_path + ".meta", "r") as f:
        metadata = json.load(f)

    encrypted_aes_key = bytes.fromhex(metadata["enc_aes_key"])
    nonce = bytes.fromhex(metadata["nonce"])
    tag = bytes.fromhex(metadata["tag"])

    rsa_key = RSA.import_key(private_key)
    cipher_rsa = PKCS1_OAEP.new(rsa_key)
    aes_key = cipher_rsa.decrypt(encrypted_aes_key)

    # Read the encrypted file (ciphertext)
    with open(enc_file_path, "rb") as f:
        ciphertext = f.read()

    # Decrypt the ciphertext using a symmetric key and previously read components
    cipher_aes = AES.new(aes_key, AES.MODE_GCM, nonce=nonce)
    plaintext = cipher_aes.decrypt_and_verify(ciphertext, tag)

    # Write the decrypted file to DEC_DIR
    dec_file_path = os.path.join(DEC_DIR, os.path.basename(enc_file_path).replace(".enc", ""))
    with open(dec_file_path, "wb") as f:
        f.write(plaintext)

    return dec_file_path

# Verify the digital signature
def verify_signature(public_key, file_path):
    rsa_key = RSA.import_key(public_key)

    # Read the file
    with open(file_path, "rb") as f:
        data = f.read()

    # Read the corresponding signature for the file
    with open(file_path + ".sig", "rb") as f:
        signature = f.read()

    # Hash the file so it can be verified
    hash = SHA256.new(data)
    try:
        pkcs1_15.new(rsa_key).verify(hash, signature)
        return True
    except ValueError:
        return False

# Encrypt all files in the defined folder
def encrypt_folder(folder_path, public_key):
    os.makedirs(ENC_DIR, exist_ok=True)

    # Loop through folder to find all files contained within
    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            # Encrypt a file using public key
            enc_file_path = encrypt_file(file_path, public_key)
            # Sign the file using private key
            sign_file(private_key, enc_file_path)
            # Print result
            print(f"Encrypted: {file_path} -> {enc_file_path}")

# Decrypt all files in the defined folder
def decrypt_folder(enc_folder, private_key, public_key):
    os.makedirs(DEC_DIR, exist_ok=True)

    # Loop through folder to find all encrypted files (.enc) to decrypt
    for file in os.listdir(enc_folder):
        if file.endswith(".enc"):
            enc_file_path = os.path.join(enc_folder, file)

            # Verify integrity before decrypting
            if verify_signature(public_key, enc_file_path):
                dec_file_path = decrypt_file(enc_file_path, private_key)
                print(f"Decrypted: {enc_file_path} -> {dec_file_path}")
            else:
                print(f"Signature verification failed for {enc_file_path}")

# Execute functions
if __name__ == "__main__":
    # Define the location of the folder to be processed
    folder_to_encrypt = "test_folder"

    # Generate RSA keys
    private_key, public_key = generate_rsa_keys()

    # Encrypt files in the folder
    encrypt_folder(folder_to_encrypt, public_key)

    # Decrypt files and verify signatures
    decrypt_folder(ENC_DIR, private_key, public_key)

Encrypted: test_folder/test_file.txt -> encrypted_files/test_file.txt.enc
Encrypted: test_folder/test_file2.txt -> encrypted_files/test_file2.txt.enc
Encrypted: test_folder/image.jpg -> encrypted_files/image.jpg.enc
Decrypted: encrypted_files/image.jpg.enc -> decrypted_files/image.jpg
Decrypted: encrypted_files/test_file2.txt.enc -> decrypted_files/test_file2.txt
Decrypted: encrypted_files/test_file.txt.enc -> decrypted_files/test_file.txt
